In [58]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from inference import get_model
import supervision as sv
from pathlib import Path
import os
from retinaface import RetinaFace
from common_tools import plt_show

# dataset images directory
root_dir = Path("/home/rahul/chalk_following_toy_neural_network_training/dataset/images")

# Blur using Supervision

This section uses Supervision to blur faces using YoloV8 model

In [2]:
root_dir = Path("/home/rahul/chalk_following_toy_neural_network_training/dataset/images")
model = get_model(model_id="yolov8n-640")

In [3]:
def pixelate_image(detection_class, image):
  pixel_annotator = sv.PixelateAnnotator()
  annotated_image = pixel_annotator.annotate(
      scene = image.copy(),
      detections=detection_class
  )
  return annotated_image

In [38]:
for image_path in root_dir.rglob("*.png"):

  image = cv2.imread(str(image_path))
  result = model.infer(image)[0]
  detection = sv.Detections.from_inference(result)
  annotated_img = pixelate_image(detection, image)
  plt_show(annotated_img)

Output hidden; open in https://colab.research.google.com to view.

# Blur using retina face

This section uses [retinaface](https://github.com/serengil/retinaface) to blur faces

In [57]:
def pixelate_face(image:np.ndarray, blocks:int=10) ->np.ndarray:

  """Taken from here: https://github.com/h0ssn1/blur-face/tree/main"""
  # divide the input image into NxN blocks
  (h, w) = image.shape[:2]
  xSteps = np.linspace(0, w, blocks + 1, dtype="int")
  ySteps = np.linspace(0, h, blocks + 1, dtype="int")
  # loop over the blocks in both the x and y direction
  for i in range(1, len(ySteps)):
      for j in range(1, len(xSteps)):
          # compute the starting and ending (x, y)-coordinates
          # for the current block
          startX = xSteps[j - 1]
          startY = ySteps[i - 1]
          endX = xSteps[j]
          endY = ySteps[i]
          # extract the ROI using NumPy array slicing, compute the
          # mean of the ROI, and then draw a rectangle with the
          # mean RGB values over the ROI in the original image
          roi = image[startY:endY, startX:endX]
          (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
          cv2.rectangle(image, (startX, startY), (endX, endY),
                        (B, G, R), -1)
  # return the pixelated blurred image
  return image

def blur(img: np.ndarray, k: int) -> np.ndarray:
    """Taken from here: https://github.com/h0ssn1/blur-face/tree/main"""
    h, w = img.shape[:2]
    print(f"img shape: {img.shape}")
    kh, kw = h // k, w // k
    if kh % 2 == 0:
      kh -= 1
    if kw % 2 == 0:
      kw -= 1
    print(f"kh: {kh}, kw: {kw}")
    img = cv2.GaussianBlur(img, ksize=(kh, kw), sigmaX=0)
    return img

def blur_face(image: np.ndarray) -> np.ndarray:
  """Uses the above 2 function & retinaface to blur faces"""
  faces = RetinaFace.detect_faces(image)

  if not faces:
    return image
  else:
    for face in faces.values():
        x1,y1,x2,y2 = face['facial_area']

        image[y1:y2, x1:x2] = pixelate_face(blur(image[y1:y2, x1:x2], 2))
    return image

In [ ]:
result_dir = root_dir.parent / Path("blured_images")
result_dir.mkdir(exist_ok=True)
for image_path in root_dir.rglob("*.png"):

    print(f" img_path: {image_path}")
    image = cv2.imread(str(image_path))
    image = blur_face(image)
    ret = cv2.imwrite(
        str(
            result_dir/ image_path.name
        ),
        image
    )
    if ret == False: raise FileExistsError
    plt_show(image)

# Dataset type conversion

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from common_tools import plt_show
root_folder = Path("/home/rahul/chalk_following_toy_neural_network_training/dataset")

In [ ]:
import shutil
images_path = root_folder / "images"
label_path = root_folder / "segmentations"

In [ ]:
def edging(
    img: np.ndarray
) -> np.ndarray:

    """Given an grayscale label image returns normalised contour
    Taken from here: https://github.com/orgs/ultralytics/discussions/8528#discussioncomment-8868637 """

    # Dilate the labels
    dilation_shape = cv2.MORPH_ELLIPSE
    dilation_size = 5
    element = cv2.getStructuringElement(dilation_shape, (2 * dilation_size + 1, 2 * dilation_size + 1),
                                        (dilation_size, dilation_size))
    dilated = cv2.dilate(img, element)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return []

    # Now, select the contour with max area or iterate through all contours
    # For example:
    contour = max(contours, key=cv2.contourArea)
    
    # Simplify contour
    epsilon = 0.001 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Convert to normalized coordinates
    height, width = img.shape
    normalized_contour = approx.reshape(-1, 2) / [width, height]

    return normalized_contour

def contour_to_str(contour):

        contour = contour.squeeze()
        class_index = 0
        return f"{class_index} " + " ".join([f"{x} {y}" for x,y in contour])

***Create an new folder with Ultralytics YOLO format:***

Taken from [here](https://docs.ultralytics.com/datasets/segment/)
The dataset label format used for training YOLO segmentation models is as follows:

One text file per image: Each image in the dataset has a corresponding text file with the same name as the image file and the ".txt" extension.
One row per object: Each row in the text file corresponds to one object instance in the image.
Object information per row: Each row contains the following information about the object instance:
Object class index: An integer representing the class of the object (e.g., 0 for person, 1 for car, etc.).
Object bounding coordinates: The bounding coordinates around the mask area, normalized to be between 0 and 1.
The format for a single row in the segmentation dataset file is as follows:


<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
In this format, <class-index> is the index of the class for the object, and <x1> <y1> <x2> <y2> ... <xn> <yn> are the bounding coordinates of the object's segmentation mask. The coordinates are separated by spaces.



In [ ]:
yolo_dataset_dir = root_folder / "yolo_format" / "temp" / "chalk"
yolo_dataset_dir.mkdir(parents = True, exist_ok = True)

In [ ]:
# populate the dir with appropriate text

for lbl_path in label_path.rglob("*.png"):

    label_img = cv2.imread(
        str(lbl_path)
    )
    contour = edging(label_img[:,:,2])
    with open(yolo_dataset_dir / ( lbl_path.stem+ ".txt" ), "w") as file:

        if len(contour) == 0:
            file.write("")
        else:
            file.write(
                contour_to_str(
                    contour
                )
            )

In [ ]:
# Copy the images from the images dir & paste it in yolo_dataset directory

for img_path in images_path.iterdir():
    shutil.copy(img_path, yolo_dataset_dir)

YAML file will be created in next section

# Train Test Val split

In [ ]:
# path for dataset generated equivalent to yolo format
yolo_dataset_dir = Path('/home/rahul/chalk_following_toy_neural_network_training/dataset/yolo_format/temp')

In [ ]:
import splitfolders


# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(
    input= yolo_dataset_dir,
    output=yolo_dataset_dir.parent / "dataset",
    seed=1337,
    ratio=(.8, .1, .1),
    group_prefix=None,
    move=False) # default values